In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import (BatchNormalization, Conv1D, Dense, Input, 
    TimeDistributed, Activation, Bidirectional, SimpleRNN, GRU, LSTM, MaxPooling1D, Dropout)


In [3]:
def simple_rnn_model(input_dim, output_dim=29):
    
    input_data = Input(name='the_input', shape=(None, input_dim))
    
    simp_rnn = GRU(output_dim, return_sequences=True, 
                 implementation=2, name='rnn')(input_data)
    
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [4]:
def rnn_model(input_dim, units, activation, output_dim=29):
    
    input_data = Input(name='the_input', shape=(None, input_dim))
    
    simp_rnn = GRU(units, activation=activation,
        return_sequences=True, implementation=2, name='rnn')(input_data)
    
    bn_rnn = BatchNormalization()(simp_rnn)
  
    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)
    
    y_pred = Activation('softmax', name='softmax')(time_dense)
  
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [5]:
def cnn_rnn_model(input_dim, filters, kernel_size, conv_stride,
    conv_border_mode, units, output_dim=29):
    
    input_data = Input(name='the_input', shape=(None, input_dim))
    
    conv_1d = Conv1D(filters, kernel_size, 
                     strides=conv_stride, 
                     padding=conv_border_mode,
                     activation='relu',
                     name='conv1d')(input_data)
    
    bn_cnn = BatchNormalization(name='bn_conv_1d')(conv_1d)
    
    simp_rnn = SimpleRNN(units, activation='relu',
        return_sequences=True, implementation=2, name='rnn')(bn_cnn)
    
    bn_rnn = BatchNormalization(name="bn_rnn")(simp_rnn)
    
    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)
    
    y_pred = Activation('softmax', name='softmax')(time_dense)
    
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: cnn_output_length(
        x, kernel_size, conv_border_mode, conv_stride)
    print(model.summary())
    return model

In [6]:
def cnn_output_length(input_length, filter_size, border_mode, stride,
                       dilation=1):
    
    if input_length is None:
        return None
    assert border_mode in {'same', 'valid'}
    dilated_filter_size = filter_size + (filter_size - 1) * (dilation - 1)
    if border_mode == 'same':
        output_length = input_length
    elif border_mode == 'valid':
        output_length = input_length - dilated_filter_size + 1
    return (output_length + stride - 1) // stride


In [7]:
def deep_rnn_model(input_dim, units, recur_layers, output_dim=29):
    
    input_data = Input(name='the_input', shape=(None, input_dim))
   
    rnn_1 = GRU(units, activation='relu',
        return_sequences=True, implementation=2, name='rnn_1')(input_data)
    
    
    
    rnn_input_for_next_rnn = rnn_1
    for layer in range(recur_layers - 1):
        layer_name = "rnn_" +  str(layer)
        rnn = GRU(units, activation="relu",return_sequences=True, 
                 implementation=2, name=layer_name) (rnn_input_for_next_rnn)
        
        batchnorm_name = "bn_" + str(layer)
        rnn_out = BatchNormalization(name=batchnorm_name)(rnn)
        rnn_input_for_next_rnn = rnn_out
        
        
    
    time_dense = TimeDistributed(Dense(output_dim))(rnn_out)
    
    y_pred = Activation('softmax', name='softmax')(time_dense)
    
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model


In [8]:
def bidirectional_rnn_model(input_dim, units, output_dim=29):
    
    input_data = Input(name='the_input', shape=(None, input_dim))
    
    bidir_rnn = Bidirectional(GRU(units, activation="relu",return_sequences=True, 
                 implementation=2, name="bidir_rnn"))(input_data)
   
    time_dense = TimeDistributed(Dense(output_dim))(bidir_rnn)
    
    y_pred = Activation('softmax', name='softmax')(time_dense)
    
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model


In [9]:
def final_model(input_dim, filters, kernel_size, conv_stride,
    conv_border_mode, units, output_dim=29):
    
   
    input_data = Input(name='the_input', shape=(None, input_dim))
    
    # CNN + maxpool
    conv_1d = Conv1D(filters, kernel_size, 
                     strides=conv_stride, 
                     padding=conv_border_mode,
                     activation='relu',
                     name='conv1d')(input_data) 
    
    
    # Batch Normalization
    conv1_normalized = BatchNormalization(name="maxpool_normalized")(conv_1d)
    
    # Bidirectionnal RNN
    bidir_rnn = Bidirectional(GRU(units, activation="relu",return_sequences=True, 
                 implementation=2, name="bidir_rnn"))(conv1_normalized)
    
    # Batch Normalization
    bidir_rnn_normalized = BatchNormalization(name="bidir_rnn_normalized")(bidir_rnn)
    
    # Time distributed
    time_dense1 = TimeDistributed(Dense(output_dim))(bidir_rnn_normalized)
    
    # Dropout
    dropout = Dropout(0.4) (time_dense1)
    
    # Time distributed
    time_dense2 = TimeDistributed(Dense(output_dim))(dropout)
    
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense2)
    
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    
    # DONE: Specify model.output_length
    model.output_length = lambda x: cnn_output_length(
        x, kernel_size, conv_border_mode, conv_stride)
    
    print(model.summary())
    return model